In [1]:
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd
import uuid

In [397]:
player_index = pd.read_csv('../updated_datasets/player_index.csv')
player_index = player_index[player_index.columns[1:]]
player_index

,Player,UUID
0,Kevin Garnett,0ed94437-a147-48ac-83f0-a3983fb029d8
1,Shaquille O'Neal,3fab405f-b6e2-4a04-8e0c-c58d6a1fea7c
2,Alonzo Mourning,24e0d59e-051e-491c-a277-266a817738ea
3,Juwan Howard,b72085ba-0fea-419f-abfe-e5f38fd489f6
4,Hakeem Olajuwon,f622472c-2cd9-4567-8b36-694b85516ebb
...,...,...
2224,Jordan Schakel,6e99656e-2a20-4aa1-9bb9-8efb2a82c3e9
2225,Jordan Goodwin,cdfbe1ba-9444-472f-9218-553857a00499
2226,Craig Sword,e4f8eb09-ae31-46e6-9673-d4b74ec31ba7
2227,Jaime Echenique,74bf6e5c-2167-42f3-b6d6-39afb49b7aa0


In [396]:
mvps = pd.read_html(requests.get('http://www.espn.com/nba/history/awards/_/id/33').text)[0]
mvps.columns = mvps.iloc[1].str.capitalize()
mvps = mvps.tail(-2)
mvps = mvps[mvps['Year'].astype('int') >= 2000]
mvps = mvps[mvps.columns[:4]]
mvps = pd.merge(left=mvps, right=player_index, how='left', on='Player')

In [395]:
dpoys = pd.read_html(requests.get('http://www.espn.com/nba/history/awards/_/id/39').text)[0]
dpoys.columns = dpoys.iloc[1].str.capitalize()
dpoys = dpoys.tail(-2)
dpoys = dpoys[dpoys['Year'].astype('int') >= 2000]
dpoys = dpoys[dpoys.columns[:4]]
dpoys = pd.merge(left=dpoys, right=player_index, how='left', on='Player')

In [398]:
def test_table(website):
    main = soup(requests.get('http://www.espn.com/nba/history/awards').text, 'html.parser')
    links = [a['href'] for a in main.find_all('a', class_='bi', href=True)]
    links.remove('//www.espn.com/nba/history/awards/_/id/34')
    
    table_list = []
    for link in links:
        #grab the header of the website page
        name = str(soup(requests.get(f'http:{link}').text).select('h2')[0]).replace(
            '<h2>', '').replace('</h2>', '').replace('<h2>', '').partition('- ')[-1].split()
        
        #'the' doesn't get iterated through so remove it
        if 'the' in name:
            name.remove('the')
            
        #set variable name to the first two capitalized words of the header
        if len(name) > 1:
            for word in name:
                if word[0] != word[0].upper():
                    name.remove(word)
            var_name = (name[0] + '_' + name[1]).lower().replace('-', '_')
        else:
            var_name = ''.join(name).lower()
            
        table_list.append(var_name)
        
        #grab the datset
        globals()[var_name] = pd.read_html(requests.get(f'http:{link}').text)[0]

        #clean the data
        globals()[var_name].columns = globals()[var_name].iloc[1].str.capitalize()
        globals()[var_name] = globals()[var_name].tail(-2)

        #some tables have blank year columns so filter for that
        globals()[var_name].fillna(method='ffill', inplace=True)

        #more cleaning
        globals()[var_name] = globals()[var_name][globals()[var_name]['Year'].astype('int') >= 2000]
        globals()[var_name] = globals()[var_name][globals()[var_name].columns[:4]]

        #merge w/ index table to add unique ids and save csvs
        globals()[var_name] = pd.merge(left=globals()[var_name], right=player_index, how='left', on='Player')
        globals()[var_name].to_csv(f'../updated_datasets/awards_data/{var_name}.csv')

    return table_list

In [390]:
test_table('http://www.espn.com/nba/history/awards')

['mvp',
 'defensive_player',
 'rookie_year',
 'sixth_man',
 'most_improved',
 'finals_mvp',
 'all_star_mvp',
 'all_nba_1st',
 'all_nba_2nd',
 'all_nba_3rd',
 'all_rookie_1st',
 'all_rookie_2nd',
 'all_defensive_1st',
 'all_defensive_2nd']

In [392]:
sixth_man

,Year,Player,Pos,Team,UUID
0,2022,Tyler Herro,PG,Miami Heat,e7af52ac-4e4e-4f88-b458-ec1a2fe18dc1
1,2021,Jordan Clarkson,PG,Utah Jazz,d650e08a-bff5-4dbc-8df0-0467a8beafb8
2,2020,Montrezl Harrell,F,LA Clippers,254d082a-656f-41a6-ac2e-fb73550cadf4
3,2019,Lou Williams,G,LA Clippers,NaN
4,2018,Lou Williams,G,LA Clippers,NaN
5,2017,Eric Gordon,G,Houston Rockets,250749af-f7de-45fb-b913-00af35d8e26c
6,2016,Jamal Crawford,G,LA Clippers,4a147dce-21fc-4703-b1a4-9b987bbbf14c
7,2015,Lou Williams,G,Toronto Raptors,NaN
8,2014,Jamal Crawford,G,LA Clippers,4a147dce-21fc-4703-b1a4-9b987bbbf14c
9,2013,JR Smith,G,New York Knicks,89cbee5a-bd3d-4df7-b18c-2ea5e6f9dce5


In [393]:
mvp

,Year,Player,Pos,Team,UUID
0,2022,Nikola Jokic,C,Denver Nuggets,23d94817-a2fe-42bd-9468-1029bfe46ce0
1,2021,Nikola Jokic,C,Denver Nuggets,23d94817-a2fe-42bd-9468-1029bfe46ce0
2,2020,Giannis Antetokounmpo,F,Milwaukee Bucks,6e925a07-b5f1-4f4f-87dd-d9fdf1d4ac44
3,2019,Giannis Antetokounmpo,F,Milwaukee Bucks,6e925a07-b5f1-4f4f-87dd-d9fdf1d4ac44
4,2018,James Harden,G,Houston Rockets,6472c887-cc8f-407b-b727-ccfcb5e803df
5,2017,Russell Westbrook,G,Oklahoma City Thunder,a17c035e-c0c2-4a2f-95ea-18f20cdb6999
6,2016,Stephen Curry,G,Golden State Warriors,0d729373-8640-4772-922d-1f8d86817adc
7,2015,Stephen Curry,G,Golden State Warriors,0d729373-8640-4772-922d-1f8d86817adc
8,2014,Kevin Durant,F,Oklahoma City Thunder,fd4f493e-5002-456a-988a-109d51120984
9,2013,LeBron James,F,Miami Heat,4ae6326c-7a7a-4902-a874-ae7a1358a285


In [394]:
defensive_player

,Year,Player,Pos,Team,UUID
0,2022,Marcus Smart,PG,Boston Celtics,f245d8cb-bd3e-416a-bd5d-f18528787dff
1,2021,Rudy Gobert,C,Utah Jazz,21dfdf6a-b6b1-4482-a3b0-ae7525f62cf5
2,2020,Giannis Antetokounmpo,F,Milwaukee Bucks,6e925a07-b5f1-4f4f-87dd-d9fdf1d4ac44
3,2019,Rudy Gobert,C,Utah Jazz,21dfdf6a-b6b1-4482-a3b0-ae7525f62cf5
4,2018,Rudy Gobert,C,Utah Jazz,21dfdf6a-b6b1-4482-a3b0-ae7525f62cf5
5,2017,Draymond Green,F,Golden State Warriors,abef788d-ee07-40a4-82f8-b4d5fc0251e2
6,2016,Kawhi Leonard,F,San Antonio Spurs,2f7f0fa9-e257-410a-9ce5-76d5169b07be
7,2015,Kawhi Leonard,F,San Antonio Spurs,2f7f0fa9-e257-410a-9ce5-76d5169b07be
8,2014,Joakim Noah,C,Chicago Bulls,d763c524-89fb-4a87-ac64-a975d7dc5b61
9,2013,Marc Gasol,C,Memphis Grizzlies,e678ec98-8743-4724-8e85-4b8f82f142a3
